In [1]:
import numpy as np

from torch.utils.data import DataLoader
from torchvision import datasets

from tqdm.notebook import tqdm

import MST
import MST.InplaceModules as inM

In [1]:
import MST.Addition.Functions as F
F.ReLU_weight_init__((2, 16))

array([[ 0.43095488, -0.14423474,  0.37419715, -0.06260999,  0.12700179,
        -0.38888735, -0.0428579 ,  0.14123613,  0.19244917, -0.08312352,
        -0.51688182,  0.28878473,  0.24633452, -0.23573606,  0.38494035,
        -0.24128614],
       [ 0.29749376, -0.20664859, -0.15795565,  0.00358781,  0.6831667 ,
        -0.05067858,  0.527228  , -0.22254494,  0.22933472, -0.57695594,
         0.38768555, -0.00072871, -0.63100801, -0.29671234,  0.16632323,
        -0.3617285 ]])

In [2]:
BATCH_SIZE = 32
NUM_WORKERS = 0

EPOCHS = 10

In [3]:
def to_np_arr(a):
    return MST.MDT_ARRAY(a) / 255

In [4]:
transform = to_np_arr

train_dataset = datasets.MNIST(
    root='datasets',
    train=True,
    transform=transform,
    download=True
)

test_dataset = datasets.MNIST(
    root='datasets',
    train=False,
    transform=transform,
    download=True
)

train_dataloader = DataLoader(
    dataset=train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=NUM_WORKERS,
)

test_dataloader = DataLoader(
    dataset=test_dataset,
    batch_size=BATCH_SIZE,
    num_workers=NUM_WORKERS,
)

In [5]:
class MyNet(MST.BasicModule):
    def __init__(self):
        super().__init__()
        self.fc1 = MST.FullyConnectedLayer(28*28, 100)
        self.fc2 = MST.FullyConnectedLayer(100, 50)
        self.fc3 = MST.FullyConnectedLayer(50, 10)

    def forward(self, x):
        x = inM.Flatten(x)
        x = self.fc1(x)
        x = inM.ReLU(x)
        x = self.fc2(x)
        x = inM.ReLU(x)
        x = self.fc3(x)
        x = inM.ReLU(x)
        return x


CELoss = MST.CrossEntropyLoss() # MSE | 
net = MyNet()

optimizer = MST.SGD(net, lr=0.01, momentum=0.9)
print(net)

MyNet:
 └── fc1: Trainable(True)
 └── fc2: Trainable(True)
 └── fc3: Trainable(True)



In [6]:
def train(net : MST.BasicModule, optimizer : MST.SGD, criterion : MST.BasicModule):
    running_loss = 0
    for ind, (images, labels) in enumerate(train_dataloader):
        images = MST.MDT_ARRAY(images)
        labels = MST.MDT_ARRAY(labels)

        output = net(images)
        loss = criterion(output, labels)
        
        loss.backward()
        optimizer.step()

        running_loss += loss
    train_loss = running_loss / len(train_dataloader)
    return train_loss


def valid(net : MST.BasicModule, criterion : MST.BasicModule):
    running_loss = 0
    correct_total = 0
    for images, labels in test_dataloader:
        images = MST.MDT_ARRAY(images)
        labels = MST.MDT_ARRAY(labels)

        output = net(images)

        loss = criterion(output, labels)
        running_loss += loss

        pred = np.argmax(output, axis=1, keepdims=True)
        correct_total += np.sum(pred[pred==labels.reshape(-1, 1)].astype(bool))
        
    rec = correct_total / len(test_dataloader.dataset)
    valid_loss = running_loss / len(test_dataloader)
    return valid_loss, rec

In [7]:
for epoch in (pbar := tqdm(range(EPOCHS))):
    train_loss = train(net, optimizer, CELoss)
    valid_loss, rec = valid(net, CELoss)

    print(f"[{epoch}] train/valid loss: {train_loss:.4f}/{valid_loss:.4f} rec: {rec:.4f}")
    pbar.set_description(f"train/valid loss: {train_loss:.4f}/{valid_loss:.4f} rec: {rec:.4f}")

  0%|          | 0/10 [00:00<?, ?it/s]

RuntimeError: It is necessary to backward first to perform optimization